In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/PATH TO THE COMPETITION DATA/train_labels_det.csv")

In [3]:
df.head()

,image_id,xmin,ymin,xmax,ymax
0,0001954c9f4a58f7ac05358b3cda8d20,0,277,345,709
1,00054819240f9d46378288b215dbcd3a,753,602,932,725
2,000dbf763348037b46558bbcb6a032ac,160,434,203,481
3,000e79208bebd8e84ce6c22fd8612a0d,70,534,211,586
4,000f13aff94499d03e3997afc55b0aa0,103,0,312,193


In [7]:
len(df["image_id"].unique())

25801

In [4]:
len(df)

25801

In [5]:
len(df["image_id"].unique())

25801

In [6]:
from PIL import Image

path = "/PATH TO THE COMPETITION DATA/images"
save_path = "/PATH TO THE COMPETITION DATA/data/images"
areas = []
for i in df.values:
    area = (i[-1] - i[-3]) * (i[-2] - i[-4]) // 1000
    areas.append(area)

In [7]:
df["area"] = areas

In [8]:
df.head()

,image_id,xmin,ymin,xmax,ymax,area
0,0001954c9f4a58f7ac05358b3cda8d20,0,277,345,709,149
1,00054819240f9d46378288b215dbcd3a,753,602,932,725,22
2,000dbf763348037b46558bbcb6a032ac,160,434,203,481,2
3,000e79208bebd8e84ce6c22fd8612a0d,70,534,211,586,7
4,000f13aff94499d03e3997afc55b0aa0,103,0,312,193,40


In [9]:
from sklearn.model_selection import StratifiedKFold

folds = df.copy()
Fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds["area"])):
    folds.loc[val_index, "fold"] = int(n)
folds["fold"] = folds["fold"].astype(int)
print(folds.groupby(["fold", "area"]).size())

fold  area
0     0       298
      1       469
      2       344
      3       264
      4       208
             ... 
4     1274      1
      1280      1
      1286      1
      1303      1
      1310     15
Length: 3129, dtype: int64


/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [12]:
folds.to_csv("folds.csv", index=False)

In [2]:
def convert_box(x1, y1, x2, y2, w, h):
    W = x2 - x1
    H = y2 - y1
    cx = x1 + W // 2
    cy = y1 + H // 2
    return cx / w, cy / h, W / w, H / h

In [24]:
path = "/PATH TO THE COMPETITION DATA/images"
save_path = "/PATH TO THE COMPETITION DATA/data/images"
### you need to make directories inside data folder as follows
# mkdir images
# cd images
# mkdir train
# mkdir val
# cd ..
# mkdir labels
# cd labels
# mkdir train
# mkdir val
# cd ..
for idx, i in enumerate(folds.values):
    im = Image.open(f"{path}/{i[0]}.png")
    w, h = im.size
    x, y, w, h = convert_box(i[1], i[2], i[3], i[4], w, h)
    if i[-1] == 0:
        im.save(f"{save_path}/val/{i[0]}.png")
        with open(f"{save_path.replace('images','labels')}/val/{i[0]}.txt", "a") as f:
            f.write(f"0 {x} {y} {w} {h}\n")
    # else:
    ## you can return the else to get the split we used to tune the parameters
    ## But the training later done on all data
    im.save(f"{save_path}/train/{i[0]}.png")
    with open(f"{save_path.replace('images','labels')}/train/{i[0]}.txt", "a") as f:
        f.write(f"0 {x} {y} {w} {h}\n")
    if idx % 1000 == 0:
        print(25000 - idx, "MORE TO GO\n")

25000 MORE TO GO

24000 MORE TO GO

23000 MORE TO GO

22000 MORE TO GO

21000 MORE TO GO

20000 MORE TO GO

19000 MORE TO GO

18000 MORE TO GO

17000 MORE TO GO

16000 MORE TO GO

15000 MORE TO GO

14000 MORE TO GO

13000 MORE TO GO

12000 MORE TO GO

11000 MORE TO GO

10000 MORE TO GO

9000 MORE TO GO

8000 MORE TO GO

7000 MORE TO GO

6000 MORE TO GO

5000 MORE TO GO

4000 MORE TO GO

3000 MORE TO GO

2000 MORE TO GO

1000 MORE TO GO

0 MORE TO GO

